In [1]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

path = r'C:\chromedriver-win64\chromedriver.exe'
service = Service(path)
service.start()

driver = webdriver.Chrome(path, options=options)
driver.get("https://coinmarketcap.com/")

scroll_pause_time = 1
scroll_step = 500 

last_height = driver.execute_script("return document.body.scrollHeight")
current_scroll = 0

while current_scroll < last_height:
    driver.execute_script(f"window.scrollBy(0, {scroll_step});")
    time.sleep(scroll_pause_time)
    current_scroll += scroll_step
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height > last_height:
        last_height = new_height

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

data = []

rows = driver.find_elements(By.XPATH, '//table[contains(@class, "cmc-table")]//tbody/tr')

for row in rows:
    cols = row.find_elements(By.TAG_NAME, 'td')
    try:
        data.append({
            "Rank": cols[1].text,
            "Name": cols[2].text,
            "Price": cols[3].text,
            "1h %": cols[4].text,
            "24h %": cols[5].text,
            "7d %": cols[6].text,
            "Market Cap": cols[7].text,
            "Volume(24h)": cols[8].text
        })
    except IndexError:
        continue 

driver.quit()

df = pd.DataFrame(data)
df.to_excel("coinmarketcap_data.xlsx", index=False)

print("✅ Data saved to 'coinmarketcap_data.xlsx'")


✅ Data saved to 'coinmarketcap_data.xlsx'
